In [ ]:
# Real life data

import logging
import threading
import itertools
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
import seaborn as seabornInstance
from sqlalchemy import Column, Integer, String, Float, DateTime, Boolean, func
from iotfunctions import base
from iotfunctions import bif
from iotfunctions import entity
from iotfunctions import metadata
from iotfunctions.metadata import EntityType
from iotfunctions.db import Database
from iotfunctions.enginelog import EngineLogging
from iotfunctions import estimator
from iotfunctions.ui import (UISingle, UIMultiItem, UIFunctionOutSingle,
                 UISingleItem, UIFunctionOutMulti, UIMulti, UIExpression,
                 UIText, UIStatusFlag, UIParameters)
from mmfunctions.anomaly import (SaliencybasedGeneralizedAnomalyScore, SpectralAnomalyScore,
                 FFTbasedGeneralizedAnomalyScore, KMeansAnomalyScore)
import datetime as dt
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import scipy as sp
import scipy.fftpack
import skimage as ski  
from skimage import util as skiutil # for nifty windowing
import pyod as pyod
from pyod.utils.data import generate_data
from pyod.utils.data import evaluate_print
from pyod.utils.example import visualize
from pyod.models.knn import KNN
from pyod.models.iforest import IForest
%matplotlib inline
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

EngineLogging.configure_console_logging(logging.INFO)

In [ ]:
# setting to make life easier
Temperature='Temperature'
kmeans='TemperatureKmeansScore'
fft='TemperatureFFTScore'
spectral='TemperatureSpectralScore'
sal='SaliencyAnomalyScore'
gen='TemperatureGeneralizedScore'
kmeansA='kmeansAnomaly'
kmeansB='kmeansAnomalyB'
spectralA='spectralAnomaly'
fftA='fftAnomaly'
salA='salAnomaly'
genA='genAnomaly'

kmeans_break=1.3
spectral_break = 2.8
fft_break = 100
sal_break = 100
gen_break = 30000



#### What will be shown

General approach is straightforward
* read raw data in
* transform it so that it is compatible to the Monitoring pipeline
* add yet another anomaly detector based on computer vision technology. The point here is to show how to run pipeline anomaly functions 'locally', an important concept for automated testing.
* simplify the dataframe - we have only one entity, no need for an entity index
* render input data and anomaly scores properly scaled

<br>

We start with Microsoft's anomaly test data found here
https://github.com/microsoft/anomalydetector/blob/master/samples/sample.csv

and then proceed to applying anomaly detection to real life pump data


<br>


#### Current inventory of anomaly detectors by type

This is the list of functions to apply


| Detector | ML Type | Type         | How does it work |
| ------- | ------------ | ------- | ---------------- |
| KMeans | Unsupervised | Proximity | Clusters data points in centroid buckets, small buckets are outliers, score is distance to closest other bucket |
| Generalized | Unsupervised | Linear Model | Covariance matrix over data point vectors serves to measure multi-dimensional deviation |
| FFT | Unsupervised | Linear Model | Run FFT before applying Generalized |
| Spectral | Unsupervised | Linear Model | Compute signal energy to reduce dimensions |
| Saliency | Unsupervised | Linear Model | Apply saliency transform (from computer vision |
| SimpleAnomaly | **Supervised** | Ensemble | Run Gradient boosting on training data, anomaly if prediction deviates from actual data |
| --- | **Supervised** | LSTM | Train a stacked LSTM, anomaly if prediction deviates from actual data |



In [ ]:
# Run on the good pump first 
# Get stuff in
df_i = pd.read_csv('./AzureAnomalysample.csv', index_col=False, parse_dates=['timestamp'])

df_i['entity']='MyRoom'
df_i['Temperature']=df_i['value'] + 20
df_i = df_i.drop(columns=['value'])

# and sort it by timestamp
df_i = df_i.sort_values(by='timestamp')
df_i = df_i.set_index(['entity','timestamp']).dropna()

df_i.head(2)

In [ ]:
# Now run the anomaly functions as if they were executed in a pipeline

spsi = SpectralAnomalyScore(Temperature, 12, spectral)
et = spsi._build_entity_type(columns = [Column(Temperature,Float())])
spsi._entity_type = et
df_i = spsi.execute(df=df_i)

sali = SaliencybasedGeneralizedAnomalyScore(Temperature, 12, sal)
et = sali._build_entity_type(columns = [Column(Temperature,Float())])
sali._entity_type = et
df_i = sali.execute(df=df_i)

ffti = FFTbasedGeneralizedAnomalyScore(Temperature, 12, fft)
et = ffti._build_entity_type(columns = [Column(Temperature,Float())])
ffti._entity_type = et
df_i = ffti.execute(df=df_i)

kmi = KMeansAnomalyScore(Temperature, 12, kmeans)
et = kmi._build_entity_type(columns = [Column(Temperature,Float())])
kmi._entity_type = et
df_i = kmi.execute(df=df_i)

df_i.describe()

In [ ]:
# Simplify our pandas dataframe to prepare input for plotting
EngineLogging.configure_console_logging(logging.INFO)

df_inputm2 = df_i.loc[['MyRoom']]
df_inputm2.reset_index(level=[0], inplace=True)

In [ ]:
# df_inputm2[spectral].values[df_inputm2[spectral] > 0.001] = 0.001
# df_inputm2[fft].values[df_inputm2[fft] < -1] = -1
df_inputm2[kmeansA] = df_inputm2[kmeans]
df_inputm2[kmeansA].values[df_inputm2[kmeansA] < kmeans_break] = np.nan
df_inputm2[kmeansA].values[df_inputm2[kmeansA] > kmeans_break] = kmeans_break
df_inputm2[kmeansB] = df_inputm2[kmeans]
df_inputm2[kmeansB].values[df_inputm2[kmeansB] >= kmeans_break] = 4
df_inputm2[kmeansB].values[df_inputm2[kmeansB] < kmeans_break] = 3

# Scale spectral and saliency 
# df_inputm2[spectral].values[df_inputm2[spectral] >= spectral_break] = -spectral_break
# df_inputm2[sal].values[df_inputm2[sal] > sal_break] = sal_break



df_inputm2[fftA] = df_inputm2[fft]
df_inputm2[fftA].values[df_inputm2[fftA] < fft_break] = np.nan
df_inputm2[fftA].values[df_inputm2[fftA] > fft_break] = fft_break
df_inputm2[spectralA] = df_inputm2[spectral]
df_inputm2[spectralA].values[df_inputm2[spectralA] < spectral_break] = np.nan
df_inputm2[spectralA].values[df_inputm2[spectralA] > spectral_break] = spectral_break
df_inputm2[salA] = df_inputm2[sal]
df_inputm2[salA].values[df_inputm2[salA] < sal_break] = np.nan
df_inputm2[salA].values[df_inputm2[salA] > sal_break] = sal_break
#df_inputm2[genA] = df_inputm2[gen]
#df_inputm2[genA].values[df_inputm2[genA] < gen_break] = np.nan
#df_inputm2[genA].values[df_inputm2[genA] > gen_break] = gen_break

plots = 5

fig, ax = plt.subplots(plots, 1, figsize=(16,24))
cnt = 0
ax[cnt].plot(df_inputm2.index, df_inputm2[Temperature]-20,linewidth=1,color='black',label=Temperature)
ax[cnt].legend(bbox_to_anchor=(1.1, 1.05))
ax[cnt].set_ylabel('Input Temperature - 20',fontsize=14,weight="bold")
cnt = 1
#ax[cnt].plot(df_inputm2.index, df_inputm2[Temperature]-20,linewidth=1,color='black',label=Input)
#ax[cnt].plot(df_inputm2.index, df_inputm2[kmeans], linewidth=2, color='magenta',label=kmeans)
#ax[cnt].plot(df_inputm2.index, df_inputm2[fft]/fft_break, linewidth=2,color='darkgreen',label=fft)
#ax[cnt].plot(df_inputm2.index, -df_inputm2[spectral]/spectral_break, linewidth=2,color='dodgerblue', label=spectral)
#ax[cnt].plot(df_inputm2.index, df_inputm2[sal]/sal_break, linewidth=2,color='chartreuse',label=sal)
#ax[cnt].legend(bbox_to_anchor=(1.1, 1.05))
#ax[cnt].set_ylabel('ALL',fontsize=14,weight="bold")
cnt = 1
ax[cnt].plot(df_inputm2.index, df_inputm2[Temperature]-20,linewidth=1,color='black',label=Temperature)
ax[cnt].plot(df_inputm2.index, df_inputm2[kmeans], linewidth=2, color='magenta',label=kmeans)
ax[cnt].plot(df_inputm2.index, df_inputm2[kmeansA], linewidth=10, color='red') #,label=kmeansA)
ax[cnt].legend(bbox_to_anchor=(1.1, 1.05))
ax[cnt].set_ylabel('KMeans \n detects chanages in "steepness"',fontsize=13)
cnt = 2
ax[cnt].plot(df_inputm2.index, df_inputm2[Temperature]-20,linewidth=1,color='black',label=Temperature)
ax[cnt].plot(df_inputm2.index, df_inputm2[fft]/fft_break, linewidth=2,color='darkgreen',label=fft)
ax[cnt].plot(df_inputm2.index, df_inputm2[fftA]/fft_break, linewidth=10, color='red') #,label=kmeansA)
ax[cnt].legend(bbox_to_anchor=(1.1, 1.05))
ax[cnt].set_ylabel('FFT \n detects frequency changes', fontsize=13)
cnt = 3
ax[cnt].plot(df_inputm2.index, df_inputm2[Temperature]-20,linewidth=1,color='black',label=Temperature)
ax[cnt].plot(df_inputm2.index, df_inputm2[spectral]/spectral_break, linewidth=2,color='dodgerblue', label=spectral)
ax[cnt].plot(df_inputm2.index, df_inputm2[spectralA]/spectral_break, linewidth=10, color='red') #,label=kmeansA)
ax[cnt].legend(bbox_to_anchor=(1.1, 1.05))
ax[cnt].set_ylabel('Spectral \n like FFT for less "CPU"\n less sensitive', fontsize=13)
cnt = 4
ax[cnt].plot(df_inputm2.index, df_inputm2[Temperature]-20,linewidth=1,color='black',label=Temperature)
ax[cnt].plot(df_inputm2.index, df_inputm2[sal]/sal_break, linewidth=2,color='chartreuse', label=sal)
ax[cnt].plot(df_inputm2.index, df_inputm2[salA]/sal_break, linewidth=10, color='red') #,label=kmeansA)
ax[cnt].legend(bbox_to_anchor=(1.1, 1.05))
ax[cnt].set_ylabel('Saliency \n like FFT, part of Azure\'s approach', fontsize=13)

for i in range(plots):
    ax[i].grid(True, color='white')
    ax[i].set_facecolor('lightgrey')

#### Results

Clear **winners** are 
* **KMeans** and 
* **FFT**.

Spectral is way too sensitive while Saliency 
doesn't detect the negative peak at 10/10 midnight


In [ ]:
# need a helper function to convert array columns to something easier
from scipy import linalg
def l2norm(df, tcol, col1, col2 = None, col3 = None):
    l2vib = []
    for index, row in df.iterrows():
        l2vib_element = linalg.norm(np.fromstring(row[col1].replace('[',' ').replace(']',''), sep = ','))**2
        if col2 is not None:
            l2vib_element = l2vib_element + \
                            linalg.norm(np.fromstring(row[col2].replace('[',' ').replace(']',''), sep = ','))**2
        if col3 is not None:
            l2vib_element = l2vib_element + \
                            linalg.norm(np.fromstring(row[col3].replace('[',' ').replace(']',''), sep = ','))**2
        l2vib.append(l2vib_element**(1/2))
    df[tcol] = np.asarray(l2vib)
    

def unrollAccel(df):
    l0,l1,l2,l3,l4=[],[],[],[],[]
    for i in df['ACCEL_POWER'].values:
        l0.append(eval(eval(i)[0]))
        l1.append(eval(eval(i)[1]))
        l2.append(eval(eval(i)[2]))
        l3.append(eval(eval(i)[3]))
        l4.append(eval(eval(i)[4]))
    df['accel_power_0'] = np.asarray(l0)
    df['accel_power_1'] = np.asarray(l1)
    df['accel_power_2'] = np.asarray(l2)
    df['accel_power_3'] = np.asarray(l3)
    df['accel_power_4'] = np.asarray(l4)
    
listAttr = ['timestamp','entity','vibrations','rms','accel_speed','accel_power_0','accel_power_1',
            'accel_power_2','accel_power_3','accel_power_4']

In [ ]:
# Now we proceed to customer data - GOOD CASE

# Get stuff in
df_input_raw = pd.read_csv('./Armstark04714B6046D5.csv', index_col=False, parse_dates=['RCV_TIMESTAMP_UTC'])
df_input_raw['entity']=df_input_raw['DEVICE_ID']
df_input_raw['timestamp']=df_input_raw['RCV_TIMESTAMP_UTC']

# and sort it by timestamp
df_input_raw = df_input_raw.sort_values(by='timestamp')
df_input_raw = df_input_raw.set_index(['entity','timestamp']).dropna()

l2norm(df_input_raw, 'vibrations', 'VIBRATIONS_XAXIS', 'VIBRATIONS_YAXIS', 'VIBRATIONS_ZAXIS')
l2norm(df_input_raw, 'rms', 'RMS_X', 'RMS_Y', 'RMS_Z')
l2norm(df_input_raw, 'accel_speed', 'ACCEL_SPEED')
unrollAccel(df_input_raw)
#l2norm(df_input_raw, 'accel_power', 'ACCEL_POWER')

df_input = df_input_raw.filter(listAttr, axis=1)
df_input_raw.describe()

In [ ]:
# Run Monitoring's anomaly detector functions

salii = SaliencybasedGeneralizedAnomalyScore('vibrations', 12, 'SaliencyVibrationScore')
et = salii._build_entity_type(columns = [Column('vibrations',Float())])
salii._entity_type = et
df_input = salii.execute(df=df_input)


spsii = SpectralAnomalyScore('vibrations', 12, 'SpectralVibrationScore')
et = spsii._build_entity_type(columns = [Column('vibrations',Float())])
spsii._entity_type = et
df_input = spsii.execute(df=df_input)

kmii = KMeansAnomalyScore('vibrations', 12, 'KMeansVibrationScore')
et = kmii._build_entity_type(columns = [Column('vibrations',Float())])
kmii._entity_type = et
df_input = kmii.execute(df=df_input)

fftii = FFTbasedGeneralizedAnomalyScore('vibrations', 12, 'FFTVibrationScore')
et = fftii._build_entity_type(columns = [Column('vibrations',Float())])
fftii._entity_type = et
df_input = fftii.execute(df=df_input)

df_input.describe()

In [ ]:
EngineLogging.configure_console_logging(logging.INFO)

df_input2 = df_input.loc[['04714B6046D5']]
df_input2.reset_index(level=[0], inplace=True)

# redefine
# setting to make life easier
Temperature='vibrations'
kmeans='KMeansVibrationScore'
fft='FFTVibrationScore'
spectral='SpectralVibrationScore'
sal='SaliencyVibrationScore'
#gen='TemperatureGeneralizedScore'
kmeansA='kmeansAnomaly'
kmeansB='kmeansAnomalyB'
spectralA='spectralAnomaly'
fftA='fftAnomaly'
salA='salAnomaly'
genA='genAnomaly'

kmeans_break=1.3
spectral_break = 2.8
fft_break = 100
sal_break = 100
gen_break = 30000

df_input2

In [ ]:
# df_input2[spectral].values[df_input2[spectral] > 0.001] = 0.001
df_input2[fft].values[df_input2[fft] < -1] = -1
df_input2[kmeansA] = df_input2[kmeans]
df_input2[kmeansA].values[df_input2[kmeansA] < kmeans_break] = np.nan
df_input2[kmeansA].values[df_input2[kmeansA] > kmeans_break] = kmeans_break
df_input2[kmeansB] = df_input2[kmeans]
df_input2[kmeansB].values[df_input2[kmeansB] >= kmeans_break] = 4
df_input2[kmeansB].values[df_input2[kmeansB] < kmeans_break] = 3



df_input2[fftA] = df_input2[fft]
df_input2[fftA].values[df_input2[fftA] < fft_break] = np.nan
df_input2[fftA].values[df_input2[fftA] > fft_break] = fft_break
df_input2[spectralA] = df_input2[spectral]
df_input2[spectralA].values[df_input2[spectralA] < spectral_break] = np.nan
df_input2[spectralA].values[df_input2[spectralA] > spectral_break] = spectral_break
df_input2[salA] = df_input2[sal]
df_input2[salA].values[df_input2[salA] < sal_break] = np.nan
df_input2[salA].values[df_input2[salA] > sal_break] = sal_break
#df_input2[genA] = df_input2[gen]
#df_input2[genA].values[df_input2[genA] < gen_break] = np.nan
#df_input2[genA].values[df_input2[genA] > gen_break] = gen_break

plots = 6

fig, ax = plt.subplots(plots, 1, figsize=(12,20))
cnt = 0
ax[cnt].plot(df_input2.index, df_input2[Temperature],linewidth=1,color='black',label=Temperature)
ax[cnt].plot(df_input2.index, df_input2[kmeans], linewidth=2, color='magenta',label=kmeans)
ax[cnt].plot(df_input2.index, df_input2[fft]/fft_break, linewidth=2,color='darkgreen',label=fft)
ax[cnt].plot(df_input2.index, df_input2[spectral]/spectral_break, linewidth=2,color='dodgerblue', label=spectral)
ax[cnt].plot(df_input2.index, df_input2[sal]/sal_break, linewidth=2,color='chartreuse',label=sal)
#ax[0].plot(df_input2.index, df_input2[gen]/gen_break, linewidth=2,color='darkviolet',label=gen)
ax[cnt].legend(bbox_to_anchor=(1.1, 1.05))
ax[cnt].set_ylabel('ALL',fontsize=14,weight="bold")
cnt=1
ax[cnt].plot(df_input2.index, df_input2[Temperature],linewidth=1,color='black',label=Temperature)
ax[cnt].plot(df_input2.index, df_input2[kmeans], linewidth=2, color='magenta',label=kmeans)
#ax[cnt].plot(df_input2.index, df_input2[kmeansB], linewidth=2, color='yellow') #label=kmeans)
ax[cnt].plot(df_input2.index, df_input2[kmeansA], linewidth=10, color='red') #,label=kmeansA)
ax[cnt].legend(bbox_to_anchor=(1.1, 1.05))
ax[cnt].set_ylabel('KMeans \n detects "steepness" changes',fontsize=13)
cnt=2
ax[cnt].plot(df_input2.index, df_input2[Temperature],linewidth=1,color='black',label=Temperature)
ax[cnt].plot(df_input2.index, df_input2[spectral]/spectral_break, linewidth=2,color='dodgerblue', label=spectral)
ax[cnt].plot(df_input2.index, df_input2[spectralA]/spectral_break, linewidth=10, color='red') #,label=kmeansA)
ax[cnt].legend(bbox_to_anchor=(1.1, 1.05))
ax[cnt].set_ylabel('Spectral \n detects spectrum changes', fontsize=13)
cnt=3
ax[cnt].plot(df_input2.index, df_input2[Temperature],linewidth=1,color='black',label=Temperature)
ax[cnt].plot(df_input2.index, df_input2[fft]/fft_break, linewidth=2,color='darkgreen',label=fft)
ax[cnt].plot(df_input2.index, df_input2[fftA]/fft_break, linewidth=10, color='red') #,label=kmeansA)
ax[cnt].legend(bbox_to_anchor=(1.1, 1.05))
ax[cnt].set_ylabel('FFT \n detects spectrum changes', fontsize=13)
cnt=4
ax[cnt].plot(df_input2.index, df_input2[Temperature],linewidth=1,color='black',label=Temperature)
ax[cnt].plot(df_input2.index, df_input2[sal]/sal_break, linewidth=2,color='chartreuse', label=sal)
ax[cnt].plot(df_input2.index, df_input2[salA]/sal_break, linewidth=10, color='red') #,label=kmeansA)
ax[cnt].legend(bbox_to_anchor=(1.1, 1.05))
ax[cnt].set_ylabel('Saliency \n (from computer vision)', fontsize=13)
cnt=5
ax[cnt].plot(df_input2.index, df_input2[Temperature],linewidth=1,color='black',label=Temperature)
#ax[cnt].plot(df_input2.index, df_input2[gen]/gen_break, linewidth=2,color='darkviolet', label=gen)
#ax[cnt].plot(df_input2.index, df_input2[genA]/gen_break, linewidth=10, color='red')
ax[cnt].legend(bbox_to_anchor=(1.1, 1.05))
ax[cnt].set_ylabel('Input data', fontsize=14)

for i in range(plots):
    ax[i].grid(True, color='white')
    ax[i].set_facecolor('lightgrey')

#ax.set_ylabel('Temperature-50')

In [ ]:
# Now we proceed to customer data - BAD CASE

# Get stuff in
df_inputb_raw = pd.read_csv('./Armstark04714B604101.csv', index_col=False, parse_dates=['RCV_TIMESTAMP_UTC'])
df_inputb_raw['entity']=df_inputb_raw['DEVICE_ID']
df_inputb_raw['timestamp']=df_inputb_raw['RCV_TIMESTAMP_UTC']

# and sort it by timestamp
df_inputb_raw = df_inputb_raw.sort_values(by='timestamp')
df_inputb_raw = df_inputb_raw.set_index(['entity','timestamp']).dropna()

l2norm(df_inputb_raw, 'vibrations', 'VIBRATIONS_XAXIS', 'VIBRATIONS_YAXIS', 'VIBRATIONS_ZAXIS')
l2norm(df_inputb_raw, 'rms', 'RMS_X', 'RMS_Y', 'RMS_Z')
l2norm(df_inputb_raw, 'accel_speed', 'ACCEL_SPEED')
unrollAccel(df_inputb_raw)
#l2norm(df_inputb, 'accel_power', 'ACCEL_POWER')

df_inputb = df_inputb_raw.filter(listAttr, axis=1)
df_inputb.head(5)

In [ ]:
# Again run Monitoring's anomaly detector functions

salii = SaliencybasedGeneralizedAnomalyScore('vibrations', 12, 'SaliencyVibrationScore')
et = salii._build_entity_type(columns = [Column('vibrations',Float())])
salii._entity_type = et
df_inputb = salii.execute(df=df_inputb)


spsii = SpectralAnomalyScore('vibrations', 12, 'SpectralVibrationScore')
et = spsii._build_entity_type(columns = [Column('vibrations',Float())])
spsii._entity_type = et
df_inputb = spsii.execute(df=df_inputb)

kmii = KMeansAnomalyScore('vibrations', 12, 'KMeansVibrationScore')
et = kmii._build_entity_type(columns = [Column('vibrations',Float())])
kmii._entity_type = et
df_inputb = kmii.execute(df=df_inputb)

fftii = FFTbasedGeneralizedAnomalyScore('vibrations', 12, 'FFTVibrationScore')
et = fftii._build_entity_type(columns = [Column('vibrations',Float())])
fftii._entity_type = et
df_inputb = fftii.execute(df=df_inputb)

df_inputb.describe()

In [ ]:
EngineLogging.configure_console_logging(logging.INFO)

df_inputb2 = df_inputb.loc[['04714B604101']]
df_inputb2.reset_index(level=[0], inplace=True)

# redefine
# setting to make life easier
Temperature='vibrations'
kmeans='KMeansVibrationScore'
fft='FFTVibrationScore'
spectral='SpectralVibrationScore'
sal='SaliencyVibrationScore'
#gen='TemperatureGeneralizedScore'
kmeansA='kmeansAnomaly'
kmeansB='kmeansAnomalyB'
spectralA='spectralAnomaly'
fftA='fftAnomaly'
salA='salAnomaly'
genA='genAnomaly'

kmeans_break=1.3
spectral_break = 2.8
fft_break = 100
sal_break = 100
gen_break = 30000

df_inputb2


In [ ]:
#df_inputb2[spectral].values[df_inputb2[spectral] > 0.001] = 0.001
df_inputb2[fft].values[df_inputb2[fft] < -1] = -1
df_inputb2[kmeansA] = df_inputb2[kmeans]
df_inputb2[kmeansA].values[df_inputb2[kmeansA] < kmeans_break] = np.nan
df_inputb2[kmeansA].values[df_inputb2[kmeansA] > kmeans_break] = kmeans_break
df_inputb2[kmeansB] = df_inputb2[kmeans]
df_inputb2[kmeansB].values[df_inputb2[kmeansB] >= kmeans_break] = 4
df_inputb2[kmeansB].values[df_inputb2[kmeansB] < kmeans_break] = 3



df_inputb2[fftA] = df_inputb2[fft]
df_inputb2[fftA].values[df_inputb2[fftA] < fft_break] = np.nan
df_inputb2[fftA].values[df_inputb2[fftA] > fft_break] = fft_break
df_inputb2[spectralA] = df_inputb2[spectral]
df_inputb2[spectralA].values[df_inputb2[spectralA] < 20] = np.nan
df_inputb2[spectralA].values[df_inputb2[spectralA] > 20] = 20
df_inputb2[salA] = df_inputb2[sal]
df_inputb2[salA].values[df_inputb2[salA] < sal_break] = np.nan
df_inputb2[salA].values[df_inputb2[salA] > sal_break] = sal_break
#df_input2[genA] = df_input2[gen]
#df_input2[genA].values[df_input2[genA] < gen_break] = np.nan
#df_input2[genA].values[df_input2[genA] > gen_break] = gen_break

plots = 6

fig, ax = plt.subplots(plots, 1, figsize=(12,20))
cnt=0
ax[cnt].plot(df_inputb2.index, df_inputb2[Temperature],linewidth=1,color='black',label=Temperature)
ax[cnt].plot(df_inputb2.index, df_inputb2[kmeans], linewidth=2, color='magenta',label=kmeans)
ax[cnt].plot(df_inputb2.index, df_inputb2[fft]/fft_break, linewidth=2,color='darkgreen',label=fft)
ax[cnt].plot(df_inputb2.index, df_inputb2[spectral]/spectral_break, linewidth=2,color='dodgerblue', label=spectral)
ax[cnt].plot(df_inputb2.index, df_inputb2[sal]/sal_break, linewidth=2,color='chartreuse',label=sal)
#ax[cnt].plot(df_input2.index, df_input2[gen]/gen_break, linewidth=2,color='darkviolet',label=gen)
ax[cnt].legend(bbox_to_anchor=(1.1, 1.05))
ax[cnt].set_ylabel('ALL',fontsize=14,weight="bold")
cnt=1
ax[cnt].plot(df_inputb2.index, df_inputb2[Temperature],linewidth=1,color='black',label=Temperature)
ax[cnt].plot(df_inputb2.index, df_inputb2[kmeans], linewidth=2, color='magenta',label=kmeans)
ax[cnt].plot(df_inputb2.index, df_inputb2[kmeansA], linewidth=10, color='red') #,label=kmeansA)
ax[cnt].legend(bbox_to_anchor=(1.1, 1.05))
ax[cnt].set_ylabel('KMeans \n detects chanages in "steepness"',fontsize=13)
cnt=2
ax[cnt].plot(df_inputb2.index, df_inputb2[Temperature],linewidth=1,color='black',label=Temperature)
ax[cnt].plot(df_inputb2.index, df_inputb2[spectral]/20, linewidth=2,color='dodgerblue', label=spectral)
ax[cnt].plot(df_inputb2.index, df_inputb2[spectralA]/20, linewidth=10, color='red') #,label=kmeansA)
ax[cnt].legend(bbox_to_anchor=(1.1, 1.05))
ax[cnt].set_ylabel('Spectral \n detects spectrum changes', fontsize=13)
cnt=3
ax[cnt].plot(df_inputb2.index, df_inputb2[Temperature],linewidth=1,color='black',label=Temperature)
ax[cnt].plot(df_inputb2.index, df_inputb2[fft]/fft_break, linewidth=2,color='darkgreen',label=fft)
ax[cnt].plot(df_inputb2.index, df_inputb2[fftA]/fft_break, linewidth=10, color='red') #,label=kmeansA)
ax[cnt].legend(bbox_to_anchor=(1.1, 1.05))
ax[cnt].set_ylabel('FFT \n detects spectrum changes', fontsize=13)
cnt=4
ax[cnt].plot(df_inputb2.index, df_inputb2[Temperature],linewidth=1,color='black',label=Temperature)
ax[cnt].plot(df_inputb2.index, df_inputb2[sal]/sal_break, linewidth=2,color='chartreuse', label=sal)
ax[cnt].plot(df_inputb2.index, df_inputb2[salA]/sal_break, linewidth=10, color='red') #,label=kmeansA)
ax[cnt].legend(bbox_to_anchor=(1.1, 1.05))
ax[cnt].set_ylabel('Saliency \n (computer vision)', fontsize=13)
cnt=5
ax[cnt].plot(df_inputb2.index, df_inputb2[Temperature],linewidth=1,color='black',label=Temperature)
#ax[cnt].plot(df_inputb2.index, df_inputb2[gen]/gen_break, linewidth=2,color='darkviolet', label=gen)
#ax[cnt].plot(df_inputb2.index, df_inputb2[genA]/gen_break, linewidth=10, color='red')
ax[cnt].legend()#bbox_to_anchor=(1.1, 1.05))
ax[cnt].set_ylabel('Input data', fontsize=13)

for i in range(plots):
    ax[i].grid(True, color='white')
    ax[i].set_facecolor('lightgrey')


#ax.set_ylabel('Temperature-50')

In [ ]:
# ROC curve FFT
from sklearn.metrics import roc_curve, auc, roc_auc_score

# before anomaly "negatives"
before_anomaly1 = dt.datetime(2020,1,7,0,0,0)  # before anomaly "negatives"
after_anomaly1 = dt.datetime(2020,1,14,0,0,0)  
before_anomaly2 = dt.datetime(2020,1,27,0,0,0) 
after_anomaly2 = dt.datetime(2020,1,31,0,0,0)

df_inputb2.loc[df_inputb2.index < before_anomaly1,'actual'] = 0
df_inputb2.loc[(df_inputb2.index > before_anomaly1) &
           (df_inputb2.index < after_anomaly1),'actual'] = 1
df_inputb2.loc[(df_inputb2.index > after_anomaly1) &
           (df_inputb2.index < before_anomaly2),'actual'] = 0
df_inputb2.loc[(df_inputb2.index > before_anomaly2) &
           (df_inputb2.index < after_anomaly2),'actual'] = 1
df_inputb2.loc[(df_inputb2.index > after_anomaly2),'actual'] = 0

np_range = np.arange(0, 151, 1)
score_list = []
fpr_list = []
tpr_list = []
roc_auc_list = []
yyy_test = df_inputb2['actual'].values
for ths in np_range:
    yyy_score_ = (np.greater(df_inputb2[fft], ths) | np.greater(df_inputb2[sal], ths)).astype(int)
    score_list.append(yyy_score_)
    fpr_, tpr_, _ = roc_curve(yyy_test, yyy_score_)
    fpr_list.append(fpr_)
    tpr_list.append(tpr_)
    roc_auc_list.append(auc(fpr_,tpr_))

    
yy_score = np.asarray(score_list)
fpr = np.asarray(fpr_list)
tpr = np.asarray(tpr_list)
roc_auc = np.asarray(roc_auc_list)

yyF_score = np.maximum(df_inputb2[fft].values, df_inputb2[sal].values)
fprF, tprF, _ = roc_curve(yyy_test, yyF_score/200)
roc_aucF = auc(fprF, tprF)


fig, ax = plt.subplots(1, 1, figsize=(8,6))
ax.plot(fprF, tprF, color='black', lw=0.5)
ax.plot(fpr[20], tpr[20], color='red',lw=1, label='ROC curve 20 (area = %0.2f)' % roc_auc[20])
ax.plot(fpr[30], tpr[30], color='turquoise',lw=1, label='ROC curve 30 (area = %0.2f)' % roc_auc[30])
ax.plot(fpr[50], tpr[50], color='orange',lw=1, label='ROC curve 50 (area = %0.2f)' % roc_auc[50])
ax.plot(fpr[75], tpr[75], color='violet',lw=1, label='ROC curve 75 (area = %0.2f)' % roc_auc[75])
ax.plot(fpr[100], tpr[100], color='green',lw=1, label='ROC curve 100 (area = %0.2f)' % roc_auc[100])
ax.plot(fpr[150], tpr[150], color='red',lw=1, label='ROC curve 150 (area = %0.2f)' % roc_auc[150])
ax.plot([0, 1], [0, 1], color='navy', lw=1, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate - lower is better, higher means false alerts')
plt.ylabel('True Positive Rate - higher is better, we detect more anomalies')
plt.title('Receiver operating characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()


#### Try out supervised methods

* Train a stacked LSTM
* Run gradient boosting


In [ ]:
# part of mmfunctions
import telemanom
from telemanom.telemanom.helpers import Config
from telemanom.telemanom.errors import Errors
import telemanom.telemanom.helpers as helpers
from telemanom.telemanom.channel import Channel
from telemanom.telemanom.modeling import Model

conf = Config("./telemanom/config.yaml")
list_attr=['vibrations','accel_power_0','accel_power_1','accel_power_2','accel_power_3','accel_power_4']
#list_attr=['vibrations','accel_power_0']
conf.dictionary['l_s'] = 250
#conf.dictionary

In [ ]:
# scale higher (*2) to make it a bit harder on the training run

tel_input = df_input[list_attr].values
tel_input = (tel_input - tel_input.mean())*2
tel_inputb = df_inputb[list_attr].values
tel_inputb = (tel_inputb - tel_inputb.mean())*2
np.save("telemanom/data/train/Armstarknew.npy", tel_input)
np.save("telemanom/data/test/Armstarknew.npy", tel_inputb)

In [ ]:
# Load data from 
device="Armstarknew"
chan = Channel(conf, device)
helpers.make_dirs(conf.use_id, conf, "./telemanom")

In [ ]:
chan.load_data("./telemanom")
# chan.train
dfA = pd.DataFrame(chan.train)
dfA.describe()

In [ ]:
# chan.train
fig, ax = plt.subplots(2, 1, figsize=(20, 7))
ax[0].plot((chan.train[:,0]-2)*2, color='blue')
ax[0].plot(chan.train[:,1]+0.5, color='green',zorder=1)
ax[0].plot(chan.train[:,2]-0.5, color='darkgreen')
ax[0].set_title('Train (good) data')
ax[1].plot((chan.test[:,0]-2)*2, color='blue')
ax[1].plot(chan.test[:,1]+0.5, color='green',zorder=1)
ax[1].plot(chan.test[:,2]-0.5, color='darkgreen')
ax[1].set_title('Test (bad) data')


In [ ]:
# producing overlapping windows of length 260 for lookback (250) and prediction (10)
chan.shape_data(chan.train, train=True)

In [ ]:
# init the Keras double stacked LSTM model
model = Model(conf, conf.use_id, chan, "./telemanom", False)

In [ ]:
# drink a coffee - training takes roughly 7 minutes
model.train_new(chan)

#### Training parameters

```
loss_metric: 'mse'    # minimize mean square error
optimizer: 'adam'     # sort of adaptive stochastic gradient descent
validation_split: 0.2 # 20% of the data is used for validating (val_loss)
dropout: 0.3          # ditch 30% of the LSTMs results when minimizing the loss function to avoid overfitting
lstm_batch_size: 64   # number of training data batches to evaluate per optimizer run to update the model’s parameters

patience: 10          # try at least 10 times to decrease val_loss smaller by ...
min_delta: 0.0003     # ... at least min_delta, else stop, so we get at least 'patience' epochs
epochs: 35            # no more than 35 passes through the entier training dataset.

l_s: 250              # lookback: num previous timesteps provided to model to predict future values
n_predictions: 10     # number of steps ahead to predict
```

This is defined in `telemanom/config.yaml`
<br>

In [ ]:
# predicting takes roughly 12 secs
model.batch_predict(chan, Path="./telemanom")

In [ ]:
# smooth the prediction error and apply exponential weights to it
errors = Errors(chan, conf, conf.use_id, "./telemanom")

#  for each overlapping window establish a threshold so that removing error points above it 
# maximizes the reduction of mean and standard deviation. Sort of an adaptive z-score 
errors.process_batches(chan)

In [ ]:
print (errors.E_seq, " \n ", errors.anom_scores)

In [ ]:
model.save("./telemanom")

In [ ]:
# How good are we doing ?

model.model.evaluate(chan.X_test, chan.y_test)

In [ ]:
#model.batch_predict(chan, Path="./telemanom", Train=True)


In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(20, 7))
#ax[0].plot(chan.y_train_hat[:8200] * 10, lw=0.2, color='green')  # to be done
ax[0].set_title('Vibration Forecast - good case')
ax[1].plot(chan.y_hat, lw=0.5, color='green')
ax[1].set_title('Vibration Forecast - bad case')

In [ ]:
npic = 1
fig, ax = plt.subplots(npic+1, 1, figsize=(20, (npic+1) * 5))
ax[npic-1].set_title('Anomaly detection - forecasting vibration based on acceleration')
ax[npic-1].plot(chan.y_test[:,0], lw=1, color='blue', label='vibration')

#ax.scatter(x_axis, temp_sal_high, lw=8, color='red')
for asc in errors.anom_scores:
    x_axis = np.arange(asc['start_idx'],asc['end_idx'],1)
    y_axis = np.zeros(asc['end_idx'] - asc['start_idx'])
    ax[npic-1].grid(True, color='white')
    ax[npic-1].set_facecolor('lightgrey')
    ax[npic-1].scatter(x_axis,y_axis+1.5, lw=5, color='red', zorder=10)
ax[npic-1].grid(True, color='white')
ax[npic-1].set_facecolor('lightgrey')
ax[npic-1].plot(abs(chan.y_hat - chan.y_test[:,0]) + 3, lw=3, color='green', label='deviation')
ax[npic-1].plot(chan.y_hat - 0.3, lw=2, color='darkgreen',label='prediction')
ax[npic-1].legend()
    
ax[npic].set_xlabel('Epoch')
ax[npic].set_ylabel('Loss')
ax[npic].set_title('Model training history')
ax[npic].plot(model.history.history['loss'])
ax[npic].plot(model.history.history['val_loss'],color='green')
ax[npic].grid(True, color='white')
ax[npic].set_facecolor('gainsboro')
ax[npic].legend(['Train', 'Validation'], loc='upper left')

#ax[1].plot(chan.test[:,0], lw=1, color='blue')
#ax[1].set_xlabel('Date')
#ax[1].set_ylabel('Compare with raw training data')


In [ ]:
# Now run gradient boosting - using lightGBM

X_train = df_input[['accel_speed','accel_power_0','accel_power_1','accel_power_2',
                    'accel_power_3','accel_power_4']].to_numpy()
y_train = df_input['vibrations'].to_numpy()

In [ ]:
import lightgbm
gbr = lightgbm.LGBMRegressor(n_estimators=4000, learning_rate=0.000001, num_leaves=40,
                           max_depth=20, random_state=42, loss='huber').fit(X_train, y_train)


In [ ]:

pred_good = gbr.predict(X_train)
rmse = metrics.mean_squared_error(y_train, pred_good)
gbscoreg = np.abs(pred_good - y_train)
print (rmse)

In [ ]:
X_bad = df_inputb[['accel_speed','accel_power_0','accel_power_1','accel_power_2',
                   'accel_power_3','accel_power_4']].to_numpy()
y_bad = df_inputb['vibrations'].to_numpy()
pred_bad = gbr.predict(X_bad) 
rmseb = metrics.mean_squared_error(y_bad, pred_bad)
gbscore = np.abs(pred_bad - y_bad)
print (rmseb)

In [ ]:
separator = 0.2
anomalygb = gbscore.copy() #(gbscore > separator) # * (separator + 0.1)
anomalygb[anomalygb <= separator] = 0
anomalygb[anomalygb > separator] = separator/2
anomalygb[anomalygb == 0] = np.nan
anomalygg = gbscoreg.copy()
anomalygg[anomalygg <= separator] = 0
anomalygg[anomalygg > separator] = separator/2
anomalygg[anomalygg == 0] = np.nan

fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, figsize=(12,14)) 
ax1.plot(y_bad, color='blue')
ax1.set_title('Bad Case - Input vibrations', fontsize=10)
ax1.set_ylabel('vibration')
ax2.plot(gbscore, color='green')
ax2.plot(anomalygb, color='red', lw=12, zorder=10)
ax2.set_ylabel('||actual-pred||')
ax2.set_title('Bad Case - deviating predictions', fontsize=10)
ax3.plot(y_train, color='blue')
ax3.set_ylabel('vibration')
ax3.set_title('Good Case - Input vibrations', fontsize=10)
ax4.plot(gbscoreg, color='green')
ax4.plot(anomalygg, color='red', lw=12, zorder=10)
ax4.set_ylabel('||actual-pred||')
ax4.set_title('Good Case - no anomalous deviation from prediction', fontsize=10)



In [ ]:
# ROC curve FFT
from sklearn.metrics import roc_curve, auc, roc_auc_score

# before anomaly "negatives"
before_anomaly1 = dt.datetime(2020,1,7,0,0,0)  # before anomaly "negatives"
after_anomaly1 = dt.datetime(2020,1,14,0,0,0)  
before_anomaly2 = dt.datetime(2020,1,27,0,0,0) 
after_anomaly2 = dt.datetime(2020,1,31,0,0,0)

df_inputb2.loc[df_inputb2.index < before_anomaly1,'actual'] = 0
df_inputb2.loc[(df_inputb2.index > before_anomaly1) &
           (df_inputb2.index < after_anomaly1),'actual'] = 1
df_inputb2.loc[(df_inputb2.index > after_anomaly1) &
           (df_inputb2.index < before_anomaly2),'actual'] = 0
df_inputb2.loc[(df_inputb2.index > before_anomaly2) &
           (df_inputb2.index < after_anomaly2),'actual'] = 1
df_inputb2.loc[(df_inputb2.index > after_anomaly2),'actual'] = 0

yyy_test = df_inputb2['actual'].values
fprFg, tprFg, _ = roc_curve(yyy_test, gbscore)
roc_aucFg = auc(fprFg, tprFg)

fig, ax = plt.subplots(1, 1, figsize=(7,5))
ax.plot(fprF, tprF, color='red', lw=2, label='ROC curve (area = %0.2f)' % roc_aucF)
ax.plot([0, 1], [0, 1], color='navy', lw=1, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate\nlower is better, higher means false alerts')
plt.ylabel('True Positive Rate\nhigher is better, we detect more anomalies')
plt.title('ROC - gradient boosting', fontsize=14)
plt.legend(loc="lower right")
plt.show()


#### Results

Gradient boosting appears to do much better

- Much shorter training time compared to the NASA model
- Prediction

To be investigated further
    
